In [1]:
from policyengine_us import Microsimulation
from policyengine_core.reforms import Reform
import pandas as pd

CTCS = [
    "ca_yctc",
    "co_ctc",
    "co_family_affordability_credit",
    "id_ctc",
    "md_ctc",
    "mn_child_and_working_families_credits",
    "mt_ctc",
    "nc_ctc",
    "nj_ctc",
    "nm_ctc",
    "ny_ctc",
    "or_ctc",
    "vt_ctc",
]

EITCS = [
    "ca_eitc",
    "co_eitc",
    "ct_eitc",
    "dc_eitc",
    "de_eitc",
    "hi_eitc",
    "ia_eitc",
    "il_eitc",
    "in_eitc",
    "ks_total_eitc",
    "la_eitc",
    "ma_eitc",
    "md_eitc",
    "me_eitc",
    "mi_eitc",
    "mo_wftc",
    "mt_eitc",
    "ne_eitc",
    "nj_eitc",
    "nm_eitc",
    "ny_eitc",
    "oh_eitc",
    "ok_eitc",
    "or_eitc",
    "ri_eitc",
    "sc_eitc",
    "ut_eitc",
    "va_eitc",
    "vt_eitc",
    "wa_working_families_tax_credit",
    "wi_earned_income_credit",
]

In [2]:
baseline = Microsimulation(dataset="cps_2022")
# Not sure we need this.
baseline.macro_cache_read = False

In [3]:
def calculate_impacts(neutralized_variables=None):
    if neutralized_variables is None:
        sim = Microsimulation(dataset="cps_2022")
    else:
        class reform(Reform):
            def apply(self):
                for var in neutralized_variables:
                    self.neutralize_variable(var)
        sim = Microsimulation(reform=reform, dataset="cps_2022")

    sim.macro_cache_read = False

    # Calculate net income
    net_income = sim.calc("household_net_income", period=2024, map_to="household")
    state_code_household = sim.calc("state_code", period=2024, map_to="household")

    # Calculate poverty impacts
    poverty = sim.calc("in_poverty", period=2024, map_to="person")
    state_code_person = sim.calc("state_code", period=2024, map_to="person")

    # Child poverty.
    child = sim.calc("is_child", period=2024, map_to="person")

    # Poverty gap.
    poverty_gap = sim.calc("poverty_gap", period=2024, map_to="household")

    # Calculate Gini index impacts
    personal_hh_equiv_income = sim.calculate("equiv_household_net_income")
    household_count_people = sim.calculate("household_count_people")
    personal_hh_equiv_income.weights *= household_count_people

    return pd.DataFrame({
        "net_income": net_income.groupby(state_code_household).sum(),
        "poverty": poverty.groupby(state_code_person).mean(),
        "child_poverty": poverty[child].groupby(state_code_person).mean(),
        "poverty_gap": poverty_gap.groupby(state_code_household).sum(),
        "gini_index": personal_hh_equiv_income.groupby(state_code_household).gini()
    })


In [4]:
# Stack four dataframes, for impacts of
# - baseline
# - EITCs
# - CTCs
# - EITCs and CTCs
stacked = pd.concat([
    calculate_impacts(),
    calculate_impacts(neutralized_variables=CTCS),
    calculate_impacts(neutralized_variables=EITCS),
    calculate_impacts(neutralized_variables=CTCS + EITCS)
], keys=["baseline", "CTCs", "EITCs", "CTCs and EITCs"])


# Assuming your original DataFrame is called 'stacked'
states = stacked.index.get_level_values(1).unique()
reform_types = ['baseline', 'CTCs', 'EITCs', 'CTCs and EITCs']

AttributeError: 'NoneType' object has no attribute 'clone'

In [ ]:
# Create an empty DataFrame to store the results
result_df = pd.DataFrame(columns=['state', 'reform_type', 'net_income_diff', 'poverty_pct_diff', 'child_poverty_pct_diff', 'poverty_gap_pct_diff', 'gini_index_pct_diff'])

for state in states:
    state_data = stacked.xs(state, level=1)
    
    for reform_tÍype in reform_types:
        if reform_type == 'baseline':
            continue
        
        net_income_diff = (state_data.loc[reform_type, 'net_income'] - state_data.loc['baseline', 'net_income']) / 1e6
        poverty_pct_diff = (state_data.loc[reform_type, 'poverty'] - state_data.loc['baseline', 'poverty']) / state_data.loc['baseline', 'poverty'] 
        child_poverty_pct_diff = (state_data.loc[reform_type, 'child_poverty'] - state_data.loc['baseline', 'child_poverty']) / state_data.loc['baseline', 'child_poverty'] 
        poverty_gap_pct_diff = (state_data.loc[reform_type, 'poverty_gap'] - state_data.loc['baseline', 'poverty_gap']) / state_data.loc['baseline', 'poverty_gap'] 
        gini_index_pct_diff = (state_data.loc[reform_type, 'gini_index'] - state_data.loc['baseline', 'gini_index']) / state_data.loc['baseline', 'gini_index'] 
        
        result_df = result_df.append({'state': state,
                                      'reform_type': reform_type,
                                      'net_income_diff': net_income_diff,
                                      'poverty_pct_diff': poverty_pct_diff,
                                      'child_poverty_pct_diff': child_poverty_pct_diff,
                                      'poverty_gap_pct_diff': poverty_gap_pct_diff,
                                      'gini_index_pct_diff': gini_index_pct_diff}, ignore_index=True)

result_df.to_csv("results.csv", index=False)